### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import csv

# Raw data file
file_to_load = "Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44

## Player Count

* Display the total number of players


In [38]:
#Total Players DataFrame
players = purchase_data.groupby(["SN"]).count()
player_counts = players["Purchase ID"].count()
pd.DataFrame({"Total Players": [player_counts]})

Total Players
0            576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [39]:
#Number of unique items
unique_items = purchase_data["Item ID"].value_counts().count()

#Total purchases
number_purchase = purchase_data["SN"].count()

#Total Revenue
total_rev = purchase_data["Price"].sum()

#Average Price
avg_price = round(purchase_data["Price"].mean(),2)

In [40]:
#Purchase Analysis Tool DataFrame
purchase_analysis_df = pd.DataFrame({"Number of Unique Items" : [unique_items],
                                    "Average Price" : [avg_price],
                                    "Number of Purchases" : [number_purchase],
                                    "Total Revenue" : [total_rev]})

#Format to $
purchase_analysis_df["Average Price"] = purchase_analysis_df["Average Price"].astype(float).map("${:}".format)
purchase_analysis_df["Total Revenue"] = purchase_analysis_df["Total Revenue"].astype(float).map("${:}".format)

purchase_analysis_df

Number of Unique Items Average Price  Number of Purchases Total Revenue
0                     183         $3.05                  780      $2379.77

## Gender Demographics

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [41]:
#Gender Demographics DataFrame
gender_df = pd.DataFrame(purchase_data.drop_duplicates("SN")["Gender"].value_counts())
gender_df.insert(0,"Percentage of Players",round((gender_df/player_counts)*100,2))
gender_df = gender_df.rename(columns = {"Gender" : "Total Count"})

# Format to %
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].astype(float).map("{:}%".format)

gender_df

Percentage of Players  Total Count
Male                                 84.03%          484
Female                               14.06%           81
Other / Non-Disclosed                 1.91%           11


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, etc. by gender


* For normalized purchasing, divide total purchase value by purchase count, by gender


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [42]:
#Gender Purchasing Analysis DataFrame
gender_analysis_df = pd.DataFrame(purchase_data["Gender"].value_counts())
gender_analysis_df = gender_analysis_df.rename(columns = {"Gender" : "Purchase Count"})

#Average purchase price
avg_price = round(purchase_data.groupby(["Gender"]).mean()["Price"],2)

#Total purchase price
total_price = round(purchase_data.groupby(["Gender"]).sum()["Price"],2)

#Insert Series into DataFrame
gender_analysis_df.insert(1,"Average Purchase Price",avg_price)
gender_analysis_df.insert(2,"Total Purchase Price",total_price)
gender_analysis_df.insert(3,"Normalized Totals",round(total_price/gender_df["Total Count"],2))

#Format to $
for i in gender_analysis_df.columns[1:]:
    gender_analysis_df[i] = gender_analysis_df[i].astype(float).map("${:}".format)
    
gender_analysis_df

Purchase Count Average Purchase Price  \
Male                              652                  $3.02   
Female                            113                   $3.2   
Other / Non-Disclosed              15                  $3.35   

                      Total Purchase Price Normalized Totals  
Male                              $1967.64             $4.07  
Female                             $361.94             $4.47  
Other / Non-Disclosed               $50.19             $4.56

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [43]:
# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [44]:
age_count = pd.cut(purchase_data["Age"], age_bins, labels=group_names).value_counts(sort=False)
age_percent = round((age_count/player_counts)*100,2)

#Age Demographics DataFrame
age_df = pd.DataFrame({
    "Percentage of Players" : age_percent,
    "Total Count" : age_count})

#Format to %
age_df["Percentage of Players"] = age_df["Percentage of Players"].astype(float).map("{:}%".format)

age_df

Percentage of Players  Total Count
<10                   3.99%           23
10-14                 4.86%           28
15-19                23.61%          136
20-24                63.37%          365
25-29                17.53%          101
30-34                12.67%           73
35-39                 7.12%           41
40+                   2.26%           13

In [45]:
#Correct percentage of players within age bin by droppping duplicate SN
purchase_data["Age Bin"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)
age_count2 = purchase_data.drop_duplicates("SN").groupby("Age Bin").count()["Purchase ID"]
age_percent2 = round(age_count2/player_counts * 100,2)

age_df2 = pd.DataFrame({
    "Percentage of Players" : age_percent2,
    "Total Count" : age_count2})

age_df2["Percentage of Players"] = age_df2["Percentage of Players"].astype(float).map("{:}%".format)

age_df2

Percentage of Players  Total Count
Age Bin                                   
<10                     2.95%           17
10-14                   3.82%           22
15-19                  18.58%          107
20-24                  44.79%          258
25-29                  13.37%           77
30-34                   9.03%           52
35-39                   5.38%           31
40+                     2.08%           12

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, etc. in the table below


* Calculate Normalized Purchasing


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [46]:
#Groupby age bins
purchase_data_age_grouby = purchase_data.groupby(pd.cut(purchase_data["Age"], age_bins, labels=group_names))

#Total purchase values
age_total_purchase = round(purchase_data_age_grouby["Price"].sum(),2)

#Total average values
age_avg_purchase = round(purchase_data_age_grouby["Price"].mean(),2)

#Normalized Totals
age_normalized_total = round(age_total_purchase/age_count2,2)

#Purchase Analysis by Age DataFrame
age_purchase_analysis_df = pd.DataFrame({
    "Purchase Count" : age_df["Total Count"],
    "Average Purchase Price" : age_avg_purchase,
    "Total Purchase Value" : age_total_purchase,
    "Normalized Totals" : age_normalized_total
})

#Format to $
for i in age_purchase_analysis_df.columns[1:]:
    age_purchase_analysis_df[i] = age_purchase_analysis_df[i].astype(float).map("${:}".format)
    
age_purchase_analysis_df

Purchase Count Average Purchase Price Total Purchase Value  \
Age                                                                 
<10                23                  $3.35               $77.13   
10-14              28                  $2.96               $82.78   
15-19             136                  $3.04              $412.89   
20-24             365                  $3.05             $1114.06   
25-29             101                   $2.9               $293.0   
30-34              73                  $2.93               $214.0   
35-39              41                   $3.6              $147.67   
40+                13                  $2.94               $38.24   

      Normalized Totals  
Age                      
<10               $4.54  
10-14             $3.76  
15-19             $3.86  
20-24             $4.32  
25-29             $3.81  
30-34             $4.12  
35-39             $4.76  
40+               $3.19

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [47]:
#Groupby SN
top_spender_groupby = purchase_data.groupby(["SN"])

#Purchase Count
top_spender_count = top_spender_groupby.count()["Purchase ID"]

#Average Purchase Price
top_spender_avg_purchase = round(top_spender_groupby.mean()["Price"],2)

#Total Purchase Value
top_spender_total_purchase = round(top_spender_groupby.sum()["Price"],2)

#Top Spenders DataFrame
top_spender_df = pd.DataFrame({
    "Purchase Count" : top_spender_count,
    "Average Purchase Price" : top_spender_avg_purchase,
    "Total Purchase Value" : top_spender_total_purchase
})

#Sort total purchase from descending order
top_spender_df = top_spender_df.sort_values(by=["Total Purchase Value"],ascending = False)

#Format to $
for i in top_spender_df.columns[1:]:
    top_spender_df[i] = top_spender_df[i].astype(float).map("${:}".format)
    
top_spender_df.head()

Purchase Count Average Purchase Price Total Purchase Value
SN                                                                     
Lisosia93                 5                  $3.79               $18.96
Idastidru52               4                  $3.86               $15.45
Chamjask73                3                  $4.61               $13.83
Iral74                    4                   $3.4               $13.62
Iskadarya95               3                  $4.37                $13.1

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [48]:
#Locate Item ID, Item Name, and Item Price
items_df = purchase_data.loc[:,["Item ID","Item Name","Price"]]
items_df.head()

Item ID                                  Item Name  Price
0      108  Extraction, Quickblade Of Trembling Hands   3.53
1      143                          Frenzied Scimitar   1.56
2       92                               Final Critic   4.88
3      100                                Blindscythe   3.27
4      131                                       Fury   1.44

In [49]:
#Groupby Items ID and Item Name
item_groupby = items_df.groupby(["Item ID","Item Name"])

#Item Purchase Counts
item_count = round(item_groupby["Price"].count(),2)

#Item Total Purchase Value
item_total = round(item_groupby["Price"].sum(),2)

#Item Price
item_price = round(item_total / item_count,2)

#Popular Item DataFrame
item_popular_df = pd.DataFrame({
    "Purchase Count" : item_count,
    "Item Price" : item_price,
    "Total Purchase Value" : item_total
})

#Format to $
for i in item_popular_df.columns[1:]:
    item_popular_df[i] = item_popular_df[i].astype(float).map("${:}".format)
    
#Sort purchase count from descending order
item_popular_df.sort_values(by = ["Purchase Count"], ascending = False).head()

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
108     Extraction, Quickblade Of Trembling Hands                  9   
82      Nirvana                                                    9   
19      Pursuit, Cudgel of Necromancy                              8   

                                                     Item Price  \
Item ID Item Name                                                 
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
145     Fiery Glass Crusader                              $4.58   
108     Extraction, Quickblade Of Trembling Hands         $3.53   
82      Nirvana                                            $4.9   
19      Pursuit, Cudgel of Necromancy                     $1.02   

                                                     Total Purchase Value  
Item ID Item Name                                                          
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
145     Fiery Glass Crusader                                       $41.22  
108     Extraction, Quickblade Of Trembling Hands                  $31.77  
82      Nirvana                                                     $44.1  
19      Pursuit, Cudgel of Necromancy                               $8.16

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [50]:
#Reset DataFrame to prevent formatting errors
item_popular_df = pd.DataFrame({
    "Purchase Count" : item_count,
    "Item Price" : item_price,
    "Total Purchase Value" : item_total
})

In [51]:
#Sort total purchase value from descending order
item_popular_df = item_popular_df.sort_values(by=["Total Purchase Value"],ascending = False)

#Format to $
for i in item_popular_df.columns[1:]:
    item_popular_df[i] = item_popular_df[i].astype(float).map("${:}".format)
    
item_popular_df.head()

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
82      Nirvana                                                    9   
145     Fiery Glass Crusader                                       9   
92      Final Critic                                               8   
103     Singed Scalpel                                             8   

                                                     Item Price  \
Item ID Item Name                                                 
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
82      Nirvana                                            $4.9   
145     Fiery Glass Crusader                              $4.58   
92      Final Critic                                      $4.88   
103     Singed Scalpel                                    $4.35   

                                                     Total Purchase Value  
Item ID Item Name                                                          
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
82      Nirvana                                                     $44.1  
145     Fiery Glass Crusader                                       $41.22  
92      Final Critic                                               $39.04  
103     Singed Scalpel                                              $34.8

In [52]:
#Another way to approach the problem by merging and droppping duplicate rows!
item_profit_df = pd.merge(item_popular_df,items_df, on = "Item ID").drop_duplicates("Item ID").set_index(["Item ID","Item Name"])

In [53]:
#Column Order
item_profit_df.loc[:,["Purchase Count","Item Price","Total Purchase Value"]].sort_values(by=["Purchase Count"],ascending = False).head()

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
82      Nirvana                                                    9   
108     Extraction, Quickblade Of Trembling Hands                  9   
60      Wolf                                                       8   

                                                     Item Price  \
Item ID Item Name                                                 
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
145     Fiery Glass Crusader                              $4.58   
82      Nirvana                                            $4.9   
108     Extraction, Quickblade Of Trembling Hands         $3.53   
60      Wolf                                              $3.54   

                                                     Total Purchase Value  
Item ID Item Name                                                          
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
145     Fiery Glass Crusader                                       $41.22  
82      Nirvana                                                     $44.1  
108     Extraction, Quickblade Of Trembling Hands                  $31.77  
60      Wolf                                                       $28.32

# Observable Trends

1. Men are predominant in this game having control over the market purchasing count and price.

2. The total count of player per age range matches that of a bell curve. Ages 20-24 has the largest player count and is the median of the age groups. Player count increases as it reaches ages 20-24 and decreases as it approaches 40+.

3. Items with a higher price have a higher purchase count. More people are willing to pay for items that have a higher price.